In [1]:
import re, numbers, numpy as np, pandas as pd
from typing import Dict

# ------------------------------------------------------------------ #
# 1.  Unit dictionaries (extended)                                   #
# ------------------------------------------------------------------ #
UNIT_SCALE = {
    "mm": 1e-3, "mm2": 1e-6, "mm3": 1e-9, "mm4": 1e-12,
    "cm": 1e-2, "cm2": 1e-4, "cm3": 1e-6, "cm4": 1e-8, "cm6": 1e-12,
    "m":  1.0,  "m2": 1.0,   "m3":  1.0,  "m4": 1.0,   "m6":   1.0,
    "mpa": 1e6, "kg/m": 1.0, "m2/m": 1.0,
}

UNIT_SI = {
    "mm": "m", "mm2": "m²", "mm3": "m³", "mm4": "m⁴",
    "cm": "m", "cm2": "m²", "cm3": "m³", "cm4": "m⁴", "cm6": "m⁶",
    "m": "m",  "m2": "m²",  "m3": "m³",  "m4": "m⁴",  "m6": "m⁶",
    "mpa": "Pa", "kg/m": "kg/m", "m2/m": "m²/m",
}

UNIT_RE = re.compile(
    r"(?i)\b("
    r"(?:[kM]?g/ ?m)|"
    r"m(?:\s*[236])?|"
    r"cm(?:\s*[2364])?|"
    r"mm(?:\s*[234])?|"
    r"MPa"
    r")\b", re.UNICODE
)

# ------------------------------------------------------------------ #
# 2.  Helpers                                                        #
# ------------------------------------------------------------------ #
_num_pat = re.compile(r"^\s*[-+]?(\d+\.?\d*|\.\d+)([eE][-+]?\d+)?\s*$")

def _is_number(x):
    if isinstance(x, numbers.Number):
        return not (isinstance(x, float) and np.isnan(x))
    if isinstance(x, str) and _num_pat.match(x.replace(",", "")):
        return True
    return False


def _find_header_row(raw, numeric_threshold=0.5, min_numeric_cols=3):
    for idx, row in raw.iterrows():
        vals = [v for v in row if pd.notna(v) and str(v).strip()]
        if not vals:
            continue
        if sum(_is_number(v) for v in vals) >= max(min_numeric_cols,
                                                  int(len(vals)*numeric_threshold)):
            return idx
    raise ValueError("Failed to detect first data row – inspect sheet.")


def _normalise_unit(token: str) -> str | None:
    t = token.lower().replace("^", "").replace("³", "3").replace("²", "2")
    t = re.sub(r"\s+", "", t)
    return t if t in UNIT_SCALE else None


def _build_column_index(header_rows):
    cols, units = [], []
    for col_vals in header_rows.T.itertuples(index=False):
        parts = [str(p).strip() for p in col_vals
                 if p and not str(p).lower().startswith("unnamed")]
        unit = None
        name_parts = []
        for p in parts:
            m = UNIT_RE.search(p)
            if m:
                unit = _normalise_unit(m.group(1))
                p = UNIT_RE.sub("", p).strip("[]() ")
            if p:
                name_parts.append(p)
        name = " ".join(name_parts).strip() or f"col_{len(cols)}"
        cols.append(f"{name} ({unit})" if unit else name)
        units.append(unit)
    return cols, units


def _clean_numeric(series: pd.Series) -> pd.Series:
    def _to_num(x):
        if isinstance(x, str):
            x = (x.replace(",", "")
                   .replace("−", "-")
                   .replace("\u2212", "-")
                   .strip())
            if x in {"", "-"}:
                return np.nan
            try:
                return float(x)
            except ValueError:
                return x
        return x
    return series.apply(_to_num)


def _apply_unit_conversions(df, cols, units):
    seen = set()
    for col, unit in zip(cols, units):
        if col in seen or col not in df.columns:
            continue
        seen.add(col)

        if unit is None or unit not in UNIT_SCALE:
            continue
        df[col] = pd.to_numeric(df[col], errors="coerce") * UNIT_SCALE[unit]
        df.rename(columns={col: col.replace(f"({unit})",
                                            f"({UNIT_SI[unit]})")},
                  inplace=True)
    return df

# ------------------------------------------------------------------ #
# 3.  Main reader                                                    #
# ------------------------------------------------------------------ #
def _normalize_profile_name(name: object) -> str:
    """
    Standardizes a profile name by converting it to a string, making it uppercase,
    removing all whitespace, and replacing superscript numbers.
    """
    if not isinstance(name, str):
        name = str(name)

    # Dictionary for replacing superscript numbers
    superscript_map = {'²': '2', '³': '3', '⁴': '4', '⁶': '6'}
    for sup, sub in superscript_map.items():
        name = name.replace(sup, sub)

    # Remove all whitespace characters (spaces, etc.) and convert to uppercase
    return re.sub(r'\s+', '', name).upper()

def read_profiles_xlsx(path: str) -> Dict[str, pd.DataFrame]:
    """
    Reads every sheet in *path*, cleans headers, converts all recognised
    units to SI, and returns {sheet_name: DataFrame}.
    """
    data = {}
    for sheet, raw in pd.ExcelFile(path).parse(None, header=None).items():
        enum_row = _find_header_row(raw)
        header_rows = raw.iloc[:enum_row].fillna("")
        cols, units = _build_column_index(header_rows)

        df = raw.iloc[enum_row + 1:].reset_index(drop=True)
        df.columns = cols
        df = df.loc[:, ~df.columns.duplicated()].copy()
        df = df.dropna(axis=1, how="all")
        df = df.apply(_clean_numeric)
        df = _apply_unit_conversions(df, cols, units)
        df = df.dropna(how="all").reset_index(drop=True)

        df.rename(columns={df.columns[0]: 'Profile'}, inplace=True)

        # --- THIS IS THE REQUIRED CHANGE ---
        # Apply the normalization function to every name in the 'Profile' column.
        # This ensures 'REC 50x30x2' becomes 'REC50X30X2', matching your test case.
        if 'Profile' in df.columns:
            df['Profile'] = df['Profile'].apply(_normalize_profile_name)
        # ------------------------------------

        data[sheet] = df.loc[:, ~df.columns.duplicated()]
    return data

# -------------------- example usage --------------------
FILE = "profiles_database.xlsx"
profiles = read_profiles_xlsx(FILE)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
profiles.keys()

dict_keys(['IPN', 'Square', 'Rectangular', 'Circular', 'L', 'UPN'])

In [10]:
profiles['IPN']

,Profile,Dimensiones d (m),bf (m),tf (m),hw (m),tw=r1 (m),Relaciones bf 2tf,hw tw,Ag (m²),Peso (kg/m),...,Qy (m³),"1,5 Sy (m³)",Zy (m³),J (m⁴),Cw (m⁶),X1 (Pa),X2 (10)-5 -2 (Pa),Acero Carga Alma Lp (m),Lr (m),F - 24 Carga Ala Sup. Lp (m)
0,IPN80,0.080,0.042,0.00590,0.0590,0.00390,3.56,15.10,0.000757,5.94,...,0.000002,0.000005,0.000005,7.100000e-09,8.750000e-11,3.281500e+10,700000.0,0.47,2.60,0.42
1,IPN100,0.100,0.050,0.00680,0.0750,0.00450,3.68,16.70,0.001060,8.34,...,0.000004,0.000007,0.000008,1.310000e-08,2.680000e-10,3.008200e+10,1000000.0,0.55,2.83,0.49
2,IPN120,0.120,0.058,0.00770,0.0920,0.00510,3.77,18.00,0.001420,11.10,...,0.000006,0.000011,0.000012,2.230000e-08,6.850000e-10,2.838200e+10,1290000.0,0.63,3.09,0.57
3,IPN140,0.140,0.066,0.00860,0.1090,0.00570,3.84,19.10,0.001820,14.30,...,0.000009,0.000016,0.000018,3.560000e-08,1.540000e-09,2.711700e+10,1560000.0,0.72,3.39,0.65
4,IPN160,0.160,0.074,0.00950,0.1250,0.00630,3.89,19.80,0.002280,17.90,...,0.000012,0.000022,0.000025,5.400000e-08,3.138000e-09,2.619000e+10,1800000.0,0.80,3.65,0.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,W6X12,0.153,0.102,0.00711,0.1210,0.00584,7.14,20.70,0.002290,17.90,...,0.000019,0.000037,0.000038,3.750000e-08,6.633000e-09,2.137400e+10,36600000.0,1.20,4.67,1.08
376,W6X9,0.150,0.100,0.00546,0.1210,0.00432,9.16,27.90,0.001729,13.40,...,0.000014,0.000027,0.000028,1.660000e-08,4.753000e-09,1.627200e+10,105000000.0,1.18,3.88,1.06
377,W5X19,0.131,0.128,0.01090,0.0889,0.00686,5.85,13.00,0.003574,28.30,...,0.000045,0.000089,0.000091,1.290000e-07,1.364200e-08,3.543900e+10,4040000.0,1.67,9.95,1.50
378,W5X16,0.127,0.127,0.00914,0.0889,0.00610,6.94,14.60,0.003019,23.80,...,0.000037,0.000074,0.000075,7.910000e-08,1.090300e-08,3.061300e+10,7280000.0,1.66,8.61,1.49


In [11]:
for key in profiles.keys():
  print(key)
  print(profiles[key].columns)
  print(profiles[key].head)

IPN
Index(['Profile', 'Dimensiones d (m)', 'bf (m)', 'tf (m)', 'hw (m)',
       'tw=r1 (m)', 'Relaciones bf 2tf', 'hw tw', 'Ag (m²)', 'Peso (kg/m)',
       'X-X Ix (m⁴)', 'Sx (m³)', 'rx (m)', 'Qx (m³)', 'Zx (m³)', 'Y-Y Iy (m⁴)',
       'Sy (m³)', 'ry (m)', 'Qy (m³)', '1,5 Sy (m³)', 'Zy (m³)', 'J (m⁴)',
       'Cw (m⁶)', 'X1 (Pa)', 'X2 (10)-5 -2 (Pa)',
       'Acero                  Carga Alma Lp (m)', 'Lr (m)',
       'F - 24                    Carga Ala Sup. Lp (m)'],
      dtype='object')
<bound method NDFrame.head of     Profile  Dimensiones d (m)  bf (m)   tf (m)  hw (m)  tw=r1 (m)  \
0     IPN80              0.080   0.042  0.00590  0.0590    0.00390   
1    IPN100              0.100   0.050  0.00680  0.0750    0.00450   
2    IPN120              0.120   0.058  0.00770  0.0920    0.00510   
3    IPN140              0.140   0.066  0.00860  0.1090    0.00570   
4    IPN160              0.160   0.074  0.00950  0.1250    0.00630   
..      ...                ...     ...      ...     ..

In [5]:
import pickle

# Save to a file
with open('profiles.pkl', 'wb') as f:
    pickle.dump(profiles, f)

In [75]:
import pandas as pd
from collections import defaultdict

# ---- master column map -----------------------------------------------------
RENAME_MAP = {
    # identical across sheets
    r'(?i)^designaci': 'shape',

    # depth / diameter / height
    r'(?i)\bdimensiones d': 'd',
    r'(?i)\bH \(m\)': 'd',
    r'(?i)\bD \(m\)': 'd',

    # flange / width
    r'(?i)\bbf': 'bf',
    r'(?i)^B \(m\)$': 'B',

    # thicknesses
    r'(?i)\btf': 'tf',
    r'(?i)\btw': 'tw',
    r'(?i)^t \(m\)$': 't',

    # other dims
    r'(?i)\bhw': 'hw',
    r'(?i)^p ': 'p',

    # area, weight
    r'(?i)^Ag ': 'Ag',
    r'(?i)^(peso|g) ': 'W',

    # section properties
    r'(?i)^Ix': 'Ix',
    r'(?i)^Iy': 'Iy',
    r'(?i)^I \(m4\)': 'Ix',
    r'(?i)^Sx': 'Sx',
    r'(?i)^Sy': 'Sy',
    r'(?i)^S \(m3\)': 'Sx',
    r'(?i)^Zx': 'Zx',
    r'(?i)^Zy': 'Zy',
    r'(?i)^Z \(m3\)': 'Zx',
    r'(?i)^rx': 'rx',
    r'(?i)^ry': 'ry',
    r'(?i)^r \(m\)': 'rx',
    r'(?i)^J ': 'J',
    r'(?i)^(Cw|C \(m3\))': 'Cw',   # C (torsion) kept as Cw for union
}

RENAME_MAP.update({
    r'(?i)^x-x ix': 'Ix',
    r'(?i)^y-y iy': 'Iy',
    r'(?i)^qx \(cm3\)': 'Qx',           # convert cm³ → m³ later
    r'(?i)^qy \(cm3\)': 'Qy',
    r'(?i)^\bi \(m4\)': 'Ix',           # CHS, use once then copy to Iy
    r'(?i)^\bs \(m3\)': 'Sx',           # CHS
    r'(?i)^\bz \(m3\)': 'Zx',           # CHS
    r'(?i)^dimensiones b': 'bf',        # Angles
    r'(?i)^radios.*r \(m\)': 'r1',
})

# full column universe we care about
MASTER_COLS = [
    'shape', 'shape_type', 'd', 'bf', 'B', 'tf', 'tw', 't', 'hw', 'p',
    'Ag', 'W', 'Ix', 'Iy', 'Sx', 'Sy', 'Zx', 'Zy', 'rx', 'ry',
    'J', 'Cw', 'Qx', 'Qy'
]

def standardise_columns(df: pd.DataFrame, shape_type: str) -> pd.DataFrame:
    """Rename columns per RENAME_MAP, drop duplicate labels, add missing cols."""
    cols = {}
    for c in df.columns:
        new = None
        for pat, repl in RENAME_MAP.items():
            if pd.Series([c]).str.contains(pat, regex=True).any():
                new = repl
                break
        cols[c] = new if new else c
    df = df.rename(columns=cols)

    # -------------- NEW: silently discard duplicate column names -------------
    df = df.loc[:, ~df.columns.duplicated()].copy()
    # -------------------------------------------------------------------------

    df.insert(0, 'shape_type', shape_type)

    # guarantee master columns order / presence
    for col in MASTER_COLS:
        if col not in df.columns:
            df[col] = pd.NA

    # -------------- NEW: make sure the drop didn't create dups again ---------
    df = df.loc[:, ~df.columns.duplicated()]
    # -------------------------------------------------------------------------

    return df[MASTER_COLS + [c for c in df.columns if c not in MASTER_COLS]]

def merge_profiles(profile_dict: dict[str, pd.DataFrame]) -> pd.DataFrame:
    """Return one dataframe with all shapes, harmonised."""
    frames = [standardise_columns(df.copy(), name)  # avoid SettingWithCopy
              for name, df in profile_dict.items()]
    merged = pd.concat(frames, ignore_index=True, sort=False)
    return merged

# ----- example --------------------------------------------------------------
profiles_merged = merge_profiles(profiles)

# 1. convert any leftover cm³ → m³ (Qx/Qy from UPN/IPN)
for col in ("Qx", "Qy"):
    if col in profiles_merged.columns:
        # strip odd characters and force numeric
        profiles_merged[col] = (
            profiles_merged[col]
            .astype(str)
            .str.replace(r"[^\d.+\-eE]", "", regex=True)   # keep digits, sign, dot, exp
            .replace("", pd.NA)
            .pipe(pd.to_numeric, errors="coerce")
            * 1e-6                                         # cm³ → m³
        )

# 2. duplicate CHS symmetry
chs = profiles_merged['shape_type'].str.contains(r'circular', case=False, na=False)
for a,b in [('Ix','Iy'), ('Sx','Sy'), ('Zx','Zy'), ('rx','ry')]:
    profiles_merged.loc[chs, b] = profiles_merged.loc[chs, a]

# 3. drop any still-duplicated labels
profiles_merged = profiles_merged.loc[:, ~profiles_merged.columns.duplicated()]
# profiles_merged.to_csv("all_profiles_si.csv", index=False)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
print(profiles_merged)


/var/folders/5b/dhfxmn8s1ld854wqz0c9l30r0000gn/T/ipykernel_75690/601468291.py:73: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if pd.Series([c]).str.contains(pat, regex=True).any():
/var/folders/5b/dhfxmn8s1ld854wqz0c9l30r0000gn/T/ipykernel_75690/601468291.py:73: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if pd.Series([c]).str.contains(pat, regex=True).any():
/var/folders/5b/dhfxmn8s1ld854wqz0c9l30r0000gn/T/ipykernel_75690/601468291.py:73: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if pd.Series([c]).str.contains(pat, regex=True).any():
/var/folders/5b/dhfxmn8s1ld854wqz0c9l30r0000gn/T/ipykernel_75690/601468291.py:73: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually ge